In [26]:
import numpy as np
from gsmmutils import ModelValidator, MyModel
from cobra.flux_analysis import pfba, flux_variability_analysis as fva
import gsmmutils
gsmmutils.DATA_PATH = "../data"
from cobra.io import to_json, save_json_model
from utils import *
import logging
from mewpy.problems import GKOProblem, GOUProblem
from mewpy.optimization import EA
logging.getLogger('cobra').setLevel(logging.CRITICAL)
logging.getLogger('gsmmutils').setLevel(logging.CRITICAL)
logging.getLogger('mewpy').setLevel(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

In [27]:
model = MyModel(r"../data/model_no_carotenoids.xml", "e_Biomass__cytop")
for demand in model.demands:
    if demand.id not in ["DM_C02094__chlo", "DM_C08601__chlo"] and 'photon' not in demand.id:
        demand.bounds = (0, 0)
model.reactions.e_Biomass__cytop.bounds = (0.14, 0.25)
model.exchanges.EX_C00011__dra.bounds = (-8.61, 1000)
model.reactions.e_Pigment__chlo.bounds = (0,0)
constraint = model.problem.Constraint(
            model.reactions.R09503_hn438__lum.flux_expression + model.reactions.R09503_hn673__lum.flux_expression,
            lb=0,
            ub=199.44)
model.add_cons_vars(constraint)
model.summary()

Loading

Reactions: 4623
Metabolites: 3737
Genes: 1690
Model loaded
Running pFBA


Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,7.08,0,0.00%
C00009__extr,EX_C00009__dra,0.0332,0,0.00%
C00011__extr,EX_C00011__dra,8.61,1,99.75%
C00059__extr,EX_C00059__dra,0.02207,0,0.00%
C00080__extr,EX_C00080__dra,0.05599,0,0.00%
C00120__extr,EX_C00120__dra,0.0008005,10,0.09%
C00205__extr,EX_C00205__dra,618.7,0,0.00%
C00244__extr,EX_C00244__dra,0.5456,0,0.00%
C00305__extr,EX_C00305__dra,0.0001543,0,0.00%
C00378__extr,EX_C00378__dra,0.0004601,12,0.06%


In [28]:
# Define the target
pfba_sol = pfba(model)
as_dict = pfba_sol.fluxes.to_dict()

PRODUCT_ID = 'DM_C02094__chlo'
BIOMASS_ID = 'e_Biomass__cytop'

# Optimization objectives
from mewpy.optimization.evaluation import  BPCY, WYIELD, BPCY_FVA
evaluator_1 = BPCY(BIOMASS_ID, PRODUCT_ID, method='lMOMA')
evaluator_2 = WYIELD(BIOMASS_ID, PRODUCT_ID)
# evaluator = BPCY_FVA(BIOMASS_ID, PRODUCT_ID, method='lMOMA', reference =as_dict)

In [23]:
problem = GKOProblem(model, fevaluation=[evaluator_1, evaluator_2], reference=as_dict, candidate_max_size=5)

In [24]:
ea = EA(problem, max_generations=100)
final_pop = ea.run()
df = ea.dataframe()
df.to_csv("gkoproblem.csv")

Building modification target list.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1690/1690 [01:05<00:00, 25.90it/s]
2024-03-26 14:29:28,916	INFO worker.py:1752 -- Started a local Ray instance.


Using 32 workers.
Running NSGAII
Eval(s)|     Worst      Best    Median   Average   Std Dev|     Worst      Best    Median   Average   Std Dev|
    100|  -0.000000  0.000000  0.000000  -0.000000  0.000000|  0.000646  0.000646  0.000646  0.000646  0.000000|
    200|  -0.000000  0.000000  0.000000  -0.000000  0.000000|  0.000646  0.000646  0.000646  0.000646  0.000000|
    300|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000646  0.000646  0.000646  0.000000|
    400|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000646  0.000646  0.000646  0.000000|
    500|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000646  0.000646  0.000646  0.000000|
    600|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000646  0.000646  0.000646  0.000000|
    700|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000646  0.000646  0.000646  0.000000|
    800|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646

(RayActor pid=1164439) /home/ecunha/anaconda3/envs/model_analysis/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
(RayActor pid=1164439)   warn(f"Solver status is '{status}'.", UserWarning)


   2500|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000646  0.000657  0.000015|
   2600|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000646  0.000659  0.000016|
   2700|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000646  0.000660  0.000016|
   2800|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000663  0.000016|
   2900|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000663  0.000016|
   3000|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000664  0.000016|
   3100|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000665  0.000016|
   3200|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000666  0.000016|
   3300|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000667  0.00001

...

(RayActor pid=1164439) /home/ecunha/anaconda3/envs/model_analysis/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
(RayActor pid=1164439)   warn(f"Solver status is '{status}'.", UserWarning)


   7700|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000674  0.000011|
   7800|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000674  0.000011|
   7900|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000674  0.000011|
   8000|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000674  0.000011|


(RayActor pid=1164999) /home/ecunha/anaconda3/envs/model_analysis/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
(RayActor pid=1164999)   warn(f"Solver status is '{status}'.", UserWarning)


   8100|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000674  0.000011|
   8200|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000674  0.000011|
   8300|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000674  0.000010|
   8400|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000674  0.000010|
   8500|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000674  0.000010|
   8600|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000674  0.000010|
   8700|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000674  0.000010|
   8800|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000674  0.000010|
   8900|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000646  0.000678  0.000678  0.000674  0.

(RayActor pid=2045704) /home/ecunha/anaconda3/envs/model_analysis/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
(RayActor pid=2045704)   warn(f"Solver status is '{status}'.", UserWarning)
(RayActor pid=2046942) /home/ecunha/anaconda3/envs/model_analysis/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'. [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RayActor pid=2046942)   warn(f"Solver status is '{status}'.", UserWarning) [repeated 4x across cluster]
(RayActor pid=2045467) /home/ecunha/anaconda3/envs/model_analysis/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'. [repeated 11x across cluster]
(RayActor pid=2045467)   warn(f"Solver status is '{status}'.", UserWar

In [25]:
df.head()

,Modification,Size,BPCY,WYIELD
0,{'KAF5827569_1': 0},1,0.0,0.000678
1,{},0,0.0,0.000646


In [29]:
problem = GOUProblem(model, fevaluation=[evaluator_1, evaluator_2], reference=as_dict, candidate_max_size=10)

In [30]:
ea = EA(problem, max_generations=100)
final_pop = ea.run()
df = ea.dataframe()
df.to_csv("gouproblem.csv")

2024-03-27 10:15:39,842	INFO worker.py:1585 -- Calling ray.init() again after it has already been called.


Using 32 workers.
Running NSGAII
Eval(s)|     Worst      Best    Median   Average   Std Dev|     Worst      Best    Median   Average   Std Dev|
    100|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.000000  0.000646  0.000000  0.000282  0.000319|
    200|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000000  0.000646  0.000646  0.000590  0.000175|
    300|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000580  0.000646  0.000646  0.000645  0.000007|
    400|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000580  0.000646  0.000646  0.000645  0.000007|
    500|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000580  0.000646  0.000646  0.000645  0.000007|
    600|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000580  0.000646  0.000646  0.000645  0.000007|
    700|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000580  0.000646  0.000646  0.000645  0.000007|
    800|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000580  0

In [34]:
df

,Modification,Size,BPCY,WYIELD
0,{'KAF5826945_1': 8},1,0.007890,0.015606
1,"{'KAF5826945_1': 8, 'KAF5836400_1': 8}",2,0.007783,0.016325
2,"{'KAF5840827_1': 4, 'KAF5836400_1': 8, 'KAF584...",4,0.007634,0.016403
